In [ ]:
import torch

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.

device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [ ]:

!pip install torch
!pip install transformers
!pip install -U -q PyDrive
!pip install datasets
!pip install seqeval
!pip install ray[tune]
!pip install transformers accelerate
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1

In [ ]:
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=15) # num_labels = 14 + 1


Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

# Importing Datasets

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

import json

# from datasets import load_dataset, concatenate_datasets, Features, Value, ClassLabel, Sequence
# dataset = load_dataset('json', data_files='NER_JUDGEMENT_WikiAnn_Format.json')

import os

import pandas as pd
train_df_judgement = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_TRAIN_JUDGEMENT_PREPROCESSED.json')
train_df_preamble = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_TRAIN_PREAMBLE_PREPROCESSED.json')

test_df_judgement = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_DEV_JUDGEMENT_PREPROCESSED.json')
test_df_preamble = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_DEV_PREAMBLE_PREPROCESSED.json')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(len(train_df_judgement['tokens'][1]))
print(len(train_df_judgement['ner_tags'][1]))

26
26


In [ ]:
mrsamples = pd.read_csv('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/train_postprocessed.csv')
mrsamples['tokens'] = mrsamples['tokens'].map(lambda x: eval(x))
mrsamples['ner_tags'] = mrsamples['ner_tags'].map(lambda x: eval(x))

In [ ]:
mrsamples

,tokens,ner_tags
0,"[that, extent.]","[0, 0]"
1,"[The, jurisdiction, granted, to, the, Tribunal...","[0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 10, 0, 0, 0, ..."
2,"[In, the, said, case,, Kailasm, ,, J.,, while,...","[0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[In, Bhinka, v., Charan, Singh,, (AIR, 1959, S...","[0, 12, 12, 12, 12, 12, 12, 12, 12, 0, 0, 0, 0..."
4,"[Thereafter,, he, shouted, and, his, aunt, Gau...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 5, 0, 0, ..."
...,...,...
6020,"[For, these, reasons, we, are, of, the, view, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
6021,"[After, seizure, what, is, to, be, done, with,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6022,"[The, first, case, referred, to, is, that, of,...","[0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 12, 12, 12, 1..."
6023,"[Appellant, Usha, Rattan, helped, appellant, C...","[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
print(len(mrsamples['tokens'][2]))
print(len(mrsamples['ner_tags'][2]))
mrsamples.reset_index(inplace=True, drop=True)

48
48


In [ ]:
print(len(train_df_judgement), len(train_df_preamble))
print(len(test_df_judgement), len(test_df_preamble))
total_data = len(train_df_judgement) + len(train_df_preamble) + len(test_df_judgement) + len(test_df_preamble)

8494 553
839 28


In [ ]:
# concatenating the judgement + preamble data
df = pd.concat([train_df_judgement, train_df_preamble])
df.reset_index(inplace=True, drop=True)
# print(df)
print(total_data)
# splitting train (70% of the total data)
split=int(total_data*0.7)
split_70=split /(len(train_df_judgement) + len(train_df_preamble))
train_df = df.sample(frac = split_70)
# train_df = df[:split]
# splitting validation (appx 20% of the total data)
val_df = df.drop(train_df.index)
# val_df = df[split:]
# test data (appx 10% of the total data)
test_df = pd.concat([test_df_judgement, test_df_preamble])

9914


In [ ]:
#Add noise

print('Before Noise Addition : Train, Test, Dev - ', len(train_df), len(test_df), len(val_df))

print(len(train_df), len(mrsamples))

noise_split = int(len(train_df)*0.25)
mrsamples = mrsamples[:noise_split]
train_df = pd.concat([train_df, mrsamples])
train_df.reset_index(inplace=True, drop=True)

print('Before Noise Addition : Train, Test, Dev - ', len(train_df), len(test_df), len(val_df))

Before Noise Addition : Train, Test, Dev -  6939 867 2108
6939 6025
Before Noise Addition : Train, Test, Dev -  8673 867 2108


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
#labels_list = ["OTHERS", "PETITIONER", "COURT", "RESPONDENT", "JUDGE", "OTHER_PERSON", "LAWYER", "DATE", "ORG", "GPE", "STATUTE", "PROVISION", "PRECEDENT", "CASE_NUMBER", "WITNESS"]
#label_encoding_dict = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}
#label_list_encoding_dict = {0: "OTHERS", 1: "PETITIONER", 2: "COURT", 3: "RESPONDENT", 4: "JUDGE", 5: "OTHER_PERSON", 6: "LAWYER", 7: "DATE", 8: "ORG", 9: "GPE", 10: "STATUTE", 11: "PROVISION", 12: "PRECEDENT", 13: "CASE_NUMBER", 14: "WITNESS"}

labels_list = [" OTHERS", " PETITIONER", " COURT", " RESPONDENT", " JUDGE", " OTHER_PERSON", " LAWYER", " DATE", " ORG", " GPE", " STATUTE", " PROVISION", " PRECEDENT", " CASE_NUMBER", " WITNESS"]
label_encoding_dict = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}
label_list_encoding_dict = {0: " OTHERS", 1: " PETITIONER", 2: " COURT", 3: " RESPONDENT", 4: " JUDGE", 5: " OTHER_PERSON", 6: " LAWYER", 7: " DATE", 8: " ORG", 9: " GPE", 10: " STATUTE", 11: " PROVISION", 12: " PRECEDENT", 13: " CASE_NUMBER", 14: " WITNESS"}

In [ ]:
def tokenize_all_labels(rows):
    tokenized_inputs = tokenizer(list(rows["tokens"]), truncation = True, is_split_into_words = True)
    labels, label_all = [], True
    for index, label in enumerate(rows["ner_tags"]):
        # print(i, label)
        prior_idx = None
        word_ids = tokenized_inputs.word_ids(batch_index = index)
        
        label_ids = []
        for current_idx in word_ids:
            if current_idx is None: label_ids.append(-100)
            elif label[current_idx] == '0': label_ids.append(0)
            elif current_idx != prior_idx: label_ids.append(label_encoding_dict[label[current_idx]])
            else: label_ids.append(label_encoding_dict[label[current_idx]] if label_all else -100)
            prior_idx = current_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
train_dataset_tokenized = train_dataset.map(tokenize_all_labels, batched=True)
val_dataset_tokenized = val_dataset.map(tokenize_all_labels, batched=True)

Map:   0%|          | 0/8673 [00:00<?, ? examples/s]

Map:   0%|          | 0/2108 [00:00<?, ? examples/s]

# Training

In [ ]:
import wandb
wandb.login()

True

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# hyperparameters
batch_size = 5
epoch = 5
learning_rate = 2.910635913133073e-05

# tokenizing and loading legalBERT model 
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=len(labels_list))
metric = load_metric("seqeval")

training_arguments = TrainingArguments(
    "eval_indian_legal_ner",
    # report_to='wandb',
    evaluation_strategy = "steps",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=1e-5,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    run_name='gold'  
)

# training on train set and evaluating on validation set
data_collator = DataCollatorForTokenClassification(tokenizer)

best_accuracy = 0
best_results = 0


def evaluate_metrics(pred_tuple):
    global best_accuracy
    global best_results
    predictions, labels = pred_tuple
    predictions = np.argmax(predictions, axis=2)
    
    actual_predictions = [[labels_list[pred_tuple] for (pred_tuple, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    actual_labels = [[labels_list[l] for (pred_tuple, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    results = metric.compute(predictions=actual_predictions, references=actual_labels)
    if best_accuracy < results["overall_accuracy"]: 
      best_results = results
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    
trainer = Trainer(
    model,
    training_arguments,
      train_dataset = train_dataset_tokenized,
      eval_dataset = val_dataset_tokenized,
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics=evaluate_metrics
)
trainer.train()
trainer.evaluate()
trainer.save_model('indian_legal_ner.model')
wandb.finish()

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initia

wandb: Currently logged in as: amansinghtha. Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.367800,0.174389,0.722423,0.790126,0.754759,0.943408
1000,0.170600,0.136532,0.765795,0.835435,0.799101,0.955187
1500,0.149600,0.153896,0.786393,0.857234,0.820287,0.956495
2000,0.105100,0.134655,0.805777,0.864501,0.834107,0.962661
2500,0.086300,0.133626,0.815533,0.876362,0.844854,0.964140
3000,0.088300,0.119730,0.818245,0.876042,0.846158,0.964210
3500,0.083900,0.116701,0.833300,0.879782,0.855910,0.966645
4000,0.052200,0.126145,0.833015,0.884377,0.857928,0.967580
4500,0.045400,0.126962,0.847788,0.886835,0.866872,0.969542
5000,0.048800,0.118522,0.849934,0.890895,0.869933,0.970457


eval/accuracy,▁▄▄▆▆▆▇▇▇█████████
eval/f1,▁▃▅▅▆▆▇▇▇▇▇▇██████
eval/loss,█▃▆▃▃▁▁▂▂▁▃▄▃▄▄▄▄▄
eval/precision,▁▃▄▅▆▆▇▇▇▇▇▇██████
eval/recall,▁▄▅▆▆▆▇▇▇▇████████
eval/runtime,▄▂▆▁▇██▁▇▄▂▆▂▁▁█▃▂
eval/samples_per_second,▅▇▃▇▂▁▁█▂▄▇▂▇██▁▆▇
eval/steps_per_second,▅▇▃▇▂▁▁█▂▄▇▂▇██▁▆▇
train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,██▇▇▆▆▅▅▄▄▄▃▃▂▂▁▁


In [ ]:
best_results

{'CASE_NUMBER': {'precision': 0.719626168224299,
  'recall': 0.8651685393258427,
  'f1': 0.7857142857142857,
  'number': 89},
 'COURT': {'precision': 0.8757396449704142,
  'recall': 0.896969696969697,
  'f1': 0.8862275449101796,
  'number': 165},
 'DATE': {'precision': 0.9536082474226805,
  'recall': 0.9736842105263158,
  'f1': 0.9635416666666667,
  'number': 190},
 'GPE': {'precision': 0.6868131868131868,
  'recall': 0.8223684210526315,
  'f1': 0.748502994011976,
  'number': 152},
 'JUDGE': {'precision': 0.75,
  'recall': 1.0,
  'f1': 0.8571428571428571,
  'number': 12},
 'LAWYER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 5},
 'ORG': {'precision': 0.538961038961039,
  'recall': 0.6287878787878788,
  'f1': 0.5804195804195804,
  'number': 132},
 'OTHERS': {'precision': 0.8762135922330098,
  'recall': 0.9009983361064892,
  'f1': 0.8884331419196062,
  'number': 2404},
 'OTHER_PERSON': {'precision': 0.897196261682243,
  'recall': 0.8888888888888888,
  'f1': 0.8930232558139535

# Evaluating Test Dataset

In [ ]:
wandb.init()
test_dataset_tokenized = test_dataset.map(tokenize_all_labels, batched=True)
test_results =trainer.evaluate(test_dataset_tokenized)
test_results

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

{'eval_loss': 0.14854097366333008,
 'eval_precision': 0.8523923444976077,
 'eval_recall': 0.8903048475762119,
 'eval_f1': 0.8709362014177462,
 'eval_accuracy': 0.9711531965681086,
 'eval_runtime': 12.632,
 'eval_samples_per_second': 68.635,
 'eval_steps_per_second': 13.775,
 'epoch': 5.0}

In [ ]:
sentence = '''Further, section 19(3) of the SG Act specifies that unless a different intention appears, rules contained in section 20 to 24 rules for ascertaining the intention of the parties as to the time at which the property and goods is to passed to the buyer.'''

tokenizer = AutoTokenizer.from_pretrained('./indian_legal_ner.model/')
input_tensor = tokenizer(sentence)['input_ids']
attention_tensor = tokenizer(sentence)['attention_mask']

model = AutoModelForTokenClassification.from_pretrained('./indian_legal_ner.model/', num_labels=len(labels_list))

predictions = model.forward(input_ids=torch.tensor(input_tensor).unsqueeze(0), attention_mask=torch.tensor(attention_tensor).unsqueeze(0))
predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
predictions = [labels_list[i] for i in predictions]

words = tokenizer.batch_decode(input_tensor)
for ner, word in zip(predictions, words):
  print(f'Token: {word} \t\t\t NER: {ner}')

Token: [CLS] 			 NER: OTHERS
Token: further 			 NER: OTHERS
Token: , 			 NER: OTHERS
Token: section 			 NER: PROVISION
Token: 19 			 NER: PROVISION
Token: ( 			 NER: PROVISION
Token: 3 			 NER: PROVISION
Token: ) 			 NER: PROVISION
Token: of 			 NER: OTHERS
Token: the 			 NER: OTHERS
Token: s 			 NER: STATUTE
Token: ##g 			 NER: STATUTE
Token: act 			 NER: STATUTE
Token: specifie 			 NER: OTHERS
Token: ##s 			 NER: OTHERS
Token: that 			 NER: OTHERS
Token: unless 			 NER: OTHERS
Token: a 			 NER: OTHERS
Token: different 			 NER: OTHERS
Token: intention 			 NER: OTHERS
Token: appears 			 NER: OTHERS
Token: , 			 NER: OTHERS
Token: rules 			 NER: OTHERS
Token: contained 			 NER: OTHERS
Token: in 			 NER: OTHERS
Token: section 			 NER: PROVISION
Token: 20 			 NER: PROVISION
Token: to 			 NER: PROVISION
Token: 24 			 NER: PROVISION
Token: rules 			 NER: OTHERS
Token: for 			 NER: OTHERS
Token: ascertain 			 NER: OTHERS
Token: ##ing 			 NER: OTHERS
Token: the 			 NER: OTHERS
Token: intentio

In [ ]:
!pip install flair==0.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached flair-0.10-py3-none-any.whl (322 kB)
  Using cached segtok-1.5.11-py3-none-any.whl (24 kB)
  Using cached mpld3-0.3-py3-none-any.whl
  Using cached sqlitedict-2.1.0.tar.gz (21 kB)
  Preparing metadata (setup.py) ... done
  Using cached Deprecated-1.2.13-py2.py3-none-any.whl (9.6 kB)
  Using cached bpemb-0.3.4-py3-none-any.whl (19 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Using cached sentencepiece-0.1.95.tar.gz (508 kB)
  Preparing metadata (setup.py) ... done
  Using cached konoha-4.6.5-py3-none-any.whl (20 kB)
  Using cached Janome-0.4.2-py2.py3-none-any.whl (19.7 MB)
  Using cached gdown-3.12.2-py3-none-any.whl
  Using cached conllu-4.5.2-py2.py3-none-any.whl (16 kB)
  Using cached more_itertools-8.8.0-py3-none-any.whl (48 kB)
  Using cached Wikipedia_API-0.5.8-py3-non

In [ ]:
import sys
import flair
import time
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.datasets import ColumnCorpus

In [ ]:
import numpy
train_df.to_csv(r'Data_Augmentation_for_Low_Resource_Indian_Legal_NER/train.csv', header=None, index=None, sep=' ', mode='a')
test_df.to_csv(r'Data_Augmentation_for_Low_Resource_Indian_Legal_NER/test.csv', header=None, index=None, sep=' ', mode='a')
val_df.to_csv(r'Data_Augmentation_for_Low_Resource_Indian_Legal_NER/dev.csv', header=None, index=None, sep=' ', mode='a')

In [ ]:
torch.save(model.state_dict(), "your_model_path.pt")

def loadCorpus(path):
    columns = {0: 'text', 1: 'ner'}
    return ColumnCorpus(path, columns,
                              train_file='train.csv',
                              test_file='test.csv',
                              dev_file='dev.csv')

datasetTest = loadCorpus('Data_Augmentation_for_Low_Resource_Indian_Legal_NER')

2023-05-17 04:08:04,493 Reading data from Data_Augmentation_for_Low_Resource_Indian_Legal_NER
2023-05-17 04:08:04,496 Train: Data_Augmentation_for_Low_Resource_Indian_Legal_NER/train.csv
2023-05-17 04:08:04,502 Dev: Data_Augmentation_for_Low_Resource_Indian_Legal_NER/dev.csv
2023-05-17 04:08:04,503 Test: Data_Augmentation_for_Low_Resource_Indian_Legal_NER/test.csv


In [ ]:
from flair.models import SequenceTagger
classifier = SequenceTagger.load('/content/drive/MyDrive/indian_legal_ner_25.model')

2023-05-17 04:08:53,148 loading file /content/drive/MyDrive/indian_legal_ner_25.model


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/flair/nn/model.py:132 in load                            │
│                                                                                                  │
│   129 │   │   │   warnings.filterwarnings("ignore")                                              │
│   130 │   │   │   # load_big_file is a workaround by https://github.com/highway11git to load m   │
│   131 │   │   │   # see https://github.com/zalandoresearch/flair/issues/351                      │
│ ❱ 132 │   │   │   f = file_utils.load_big_file(str(model_file))                                  │
│   133 │   │   │   state = torch.load(f, map_location='cpu')                                      │
│   134 │   │                                                                                      │
│   135 │   │   model = cls._init_model_with_state_dict(state)                                     │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/flair/file_utils.py:33 in load_big_file                  │
│                                                                                                  │
│    30 │   :return:                                                                               │
│    31 │   """                                                                                    │
│    32 │   logger.info(f"loading file {f}")                                                       │
│ ❱  33 │   with open(f, "rb") as f_in:                                                            │
│    34 │   │   # mmap seems to be much more memory efficient                                      │
│    35 │   │   bf = mmap.mmap(f_in.fileno(), 0, access=mmap.ACCESS_READ)                          │
│    36 │   │   f_in.close()                                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/indian_legal_ner_25.model'